In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Configure plots
plt.style.use("seaborn-v0_8")
sns.set_theme()

# Resolve project root (2 levels above this notebook)
PROJECT_ROOT = Path().resolve().parents[0]
DATA_PROCESSED = PROJECT_ROOT / "Data" / "processed"

# Load cleaned data
# Explicitly cast mixed string columns so pandas doesn't infer inconsistent dtypes when reloading processed CSVs.
dtype_map = {"StateHoliday": "string", "PromoInterval": "string"}
train = pd.read_csv(DATA_PROCESSED / "train_clean.csv", dtype=dtype_map)
test = pd.read_csv(DATA_PROCESSED / "test_clean.csv", dtype=dtype_map)

print("Loaded processed datasets:")
print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()

In [ ]:
# Quick check: geo and state-level features present
[col for col in train.columns if
 "Lat" in col or "Elevation" in col or "Population" in col or "Distance" in col or "State" in col]

In [ ]:
# Step 2: Structural Overview

print("Column names:")
print(train.columns.tolist())

print("\nData types:")
print(train.dtypes)

print("\nMissing values per column:")
print(train.isna().sum())

print("\nMissing values (%):")
print((train.isna().mean() * 100).round(2))

# Quick classify numerical vs categorical
numeric_cols = train.select_dtypes(include=["number"]).columns.tolist()
categorical_cols = train.select_dtypes(include=["object"]).columns.tolist()

print("\nNumeric columns:", numeric_cols)
print("Categorical columns:", categorical_cols)

In [ ]:
# Step 3: Outlier Analysis


def detect_outliers_iqr(series):
    """Return lower bound, upper bound, and boolean mask for outliers using IQR."""
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1

    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    mask = (series < lower) | (series > upper)
    return lower, upper, mask


# Analyze Sales
sales_lower, sales_upper, sales_outliers = detect_outliers_iqr(train["Sales"])

print("Sales outlier thresholds:")
print("Lower bound:", sales_lower)
print("Upper bound:", sales_upper)
print("Outlier count:", sales_outliers.sum())
print("Outlier percentage:", round(sales_outliers.mean() * 100, 2), "%")

# Analyze Customers
cust_lower, cust_upper, cust_outliers = detect_outliers_iqr(train["Customers"])

print("\nCustomers outlier thresholds:")
print("Lower bound:", cust_lower)
print("Upper bound:", cust_upper)
print("Outlier count:", cust_outliers.sum())
print("Outlier percentage:", round(cust_outliers.mean() * 100, 2), "%")

# Boxplots to visualize distributions and outliers
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sns.boxplot(data=train, y="Sales", ax=axes[0])
axes[0].set_title("Sales Outliers (Boxplot)")

sns.boxplot(data=train, y="Customers", ax=axes[1])
axes[1].set_title("Customers Outliers (Boxplot)")

plt.tight_layout()
plt.show()

In [ ]:
# Step 4: Time-based EDA

# Ensure Date column is datetime (safety)
train["Date"] = pd.to_datetime(train["Date"])

# Extract time features
train["Year"] = train["Date"].dt.year
train["Month"] = train["Date"].dt.month
train["Day"] = train["Date"].dt.day
train["WeekOfYear"] = train["Date"].dt.isocalendar().week
train["DayOfWeek"] = train["Date"].dt.dayofweek  # 0=Mon, 6=Sun

print("Extracted time features: Year, Month, Day, WeekOfYear, DayOfWeek")

train[["Date", "Year", "Month", "Day", "WeekOfYear", "DayOfWeek"]].head()

In [ ]:
# Visualize time patterns

# Sales per Month
plt.figure(figsize=(12, 5))
sns.lineplot(data=train.groupby("Month")["Sales"].mean())
plt.title("Average Sales per Month")
plt.ylabel("Average Sales")
plt.xlabel("Month")
plt.grid(True)
plt.show()

# Sales per Day of Week
plt.figure(figsize=(12, 5))
sns.barplot(x="DayOfWeek", y="Sales", data=train)
plt.title("Sales per Day of Week")
plt.xlabel("Day of Week (0=Mon, 6=Sun)")
plt.ylabel("Average Sales")
plt.show()

# Sales Trend: Year over Year
plt.figure(figsize=(12, 5))
sns.lineplot(data=train.groupby("Year")["Sales"].mean())
plt.title("Year-over-Year Sales Trend")
plt.ylabel("Average Sales")
plt.xlabel("Year")
plt.grid(True)
plt.show()

# Sales over Time (full timeseries)
plt.figure(figsize=(16, 5))
sns.lineplot(data=train, x="Date", y="Sales", alpha=0.5)
plt.title("Sales Over Time")
plt.ylabel("Sales")
plt.xlabel("Date")
plt.show()

# Average Sales per State
plt.figure(figsize=(12, 6))
sns.barplot(
    data=train,
    x="State",
    y="Sales",
    estimator="mean",
    errorbar=None
)
plt.title("Average Sales per State")
plt.xlabel("State")
plt.ylabel("Average Sales")
plt.show()

In [ ]:
# Step 5: Outlier Treatment - clipping using 1st and 99th percentiles

def clip_percentile(series, lower_q=0.01, upper_q=0.99):
    lower = series.quantile(lower_q)
    upper = series.quantile(upper_q)
    return series.clip(lower, upper)


train = train.copy()
for col in numeric_cols:
    train[col] = clip_percentile(train[col])

print("Applied percentile-based clipping (1 percent to 99 percent).")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sns.boxplot(data=train, y="Sales", ax=axes[0])
axes[0].set_title("Before Clipping")

sns.boxplot(data=train_clipped_p99, y="Sales", ax=axes[1])
axes[1].set_title("After Percentile Clipping (1 percent - 99 percent)")

plt.tight_layout()
plt.show()

In [ ]:
# Step 6: Advanced visualizations with geo and state features

import plotly.express as px

# Interactive Geo Map: Average Sales by State

state_geo = (
    train.groupby("State")
    .agg({
        "LatMean": "mean",
        "LonMean": "mean",
        "Sales": "mean",
    })
    .reset_index()
)

fig = px.scatter_map(
    state_geo,
    lat="LatMean",
    lon="LonMean",
    size="Sales",
    color="Sales",
    hover_name="State",
    color_continuous_scale="Viridis",
    zoom=4,
    height=600,
    title="Average Sales by State (Interactive Geo Map)"
)

fig.show()

In [ ]:
# Heatmap: Average Sales by State and Month

# Ensure Month exists (in case kernel was restarted or data reloaded)
train["Date"] = pd.to_datetime(train["Date"])
train["Month"] = train["Date"].dt.month

pivot_state_month = train.pivot_table(
    values="Sales",
    index="State",
    columns="Month",
    aggfunc="mean"
)

plt.figure(figsize=(12, 7))
sns.heatmap(
    pivot_state_month,
    cmap="viridis",
    linewidths=0.5
)
plt.title("Average Sales by State and Month")
plt.xlabel("Month")
plt.ylabel("State")
plt.show()

In [ ]:
import plotly.express as px

# Aggregate total sales per state
state_sales = (
    train.groupby("State")["Sales"]
    .sum()
    .reset_index()
    .sort_values("Sales", ascending=False)
)

fig = px.pie(
    state_sales,
    names="State",
    values="Sales",
    title="Share of Total Sales by State",
    hole=0.3,  # donut chart for readability
)

fig.update_traces(textposition="inside", textinfo="percent+label")

fig.update_layout(height=600)

fig.show()